# Gaussian Ranges
A simple example of ...


### Links
http://infernet.azurewebsites.net/default.aspx    
http://infernet.azurewebsites.net/docs/Two%20coins%20tutorial.aspx  
https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/Model_Learner_Pattern_POPL_Rome.pdf  

## Load & open packages

Latest download from:   
http://infernet.azurewebsites.net/download.aspx    
https://www.microsoft.com/en-us/download/details.aspx?id=52476    
Infer.NET 2.7 March 2018

In [1]:
//#load "Paket.fsx"

Expression evaluation failed: Non-negative number required.
Parameter name: count
ArgumentOutOfRangeExceptionNon-negative number required.
Parameter name: count
  at System.IO.BinaryReader.ReadBytes (System.Int32 count) [0x00013] in <77c9551943624fd18301ba6f78a841e5>:0 
  at Mono.Cecil.PE.ImageReader.ReadDebugHeader () [0x000f7] in <2e24b151dd8041d8b7d36a855bb53c66>:0 
  at Mono.Cecil.PE.ImageReader.ReadImage () [0x000a8] in <2e24b151dd8041d8b7d36a855bb53c66>:0 
  at Mono.Cecil.PE.ImageReader.ReadImage (Mono.Disposable`1[T] stream, System.String file_name) [0x00007] in <2e24b151dd8041d8b7d36a855bb53c66>:0 
  at Mono.Cecil.ModuleDefinition.ReadModule (Mono.Disposable`1[T] stream, System.String fileName, Mono.Cecil.ReaderParameters parameters) [0x00006] in <2e24b151dd8041d8b7d36a855bb53c66>:0 
  at Mono.Cecil.ModuleDefinition.ReadModule (System.String fileName, Mono.Cecil.ReaderParameters parameters) [0x0006c] in <2e24b151dd8041d8b7d36a855bb53c66>:0 
  at Mono.Cecil.ModuleDefinition.Read

In [13]:
//Paket.Package
//  [ "Infer.NET"
//  ]

### Unzip full distribution to the file system

This solution due to Tomas Petricek.  
First, download the ZIP file release:  
(From https://www.microsoft.com/en-us/download/confirmation.aspx?id=52476)

In [ ]:
let wc = new System.Net.WebClient()
wc.DownloadFile(
    "https://download.microsoft.com/download/D/C/4/DC44E915-26FE-48AB-BD7C-AC1D92B23230/Infer.NET%202.7.1803.1901.zip", 
    "/home/nbuser/IfSharp/bin/packages/infer.zip")
//Now, unzip it – we could get .NET ZipFile to do this, but invoking the Linux command line tools works.
open System.Diagnostics
let ps = 
    ProcessStartInfo(
        FileName="unzip", 
        Arguments="/home/nbuser/IfSharp/bin/packages/infer.zip -d /home/nbuser/IfSharp/bin/packages/infer",
        RedirectStandardOutput=true,UseShellExecute=false)
let p = Process.Start(ps)
p.StandardOutput.ReadToEnd()

### Load & open packages

In [1]:
#I "/home/nbuser/IfSharp/bin/packages/infer/Bin/"
#r "Infer.FSharp"
#r "Infer.Runtime"
#r "Infer.Compiler"

In [2]:
open System
open MicrosoftResearch.Infer
open MicrosoftResearch.Infer.Models
open MicrosoftResearch.Infer.Distributions
open MicrosoftResearch.Infer.Factors
open MicrosoftResearch.Infer.Maths
open MicrosoftResearch.Infer.FSharp
open MicrosoftResearch.Infer.FSharp.Variable

## Model

Source: `\Infer.NET 2.7.1803.1901\Samples\F#\TestFSharp`

Examples include:
* TwoCoins
* TruncatedGaussian
* GaussianRanges
* ClinicalTrial
* BayesPoint
* MixtureGaussians

### The model

In [3]:
let len = Variable.New<int>()
let dataRange = Range(len)
let mean = Variable.GaussianFromMeanAndVariance(0.0, 100.0)
let precision = Variable.GammaFromShapeAndScale(1.0, 1.0)
let x = Variable.AssignVariableArray 
            (Variable.Array<float>(dataRange))  
             dataRange (fun d -> Variable.GaussianFromMeanAndPrecision(mean, precision))

### The data

In [4]:
let data = Array.init 100 (fun _ -> Rand.Normal(0.0, 1.0))

### Binding the data

In [5]:
len.ObservedValue <- data.Length
x.ObservedValue <- data

### The inference

In [6]:
let ie = InferenceEngine(VariationalMessagePassing())
printfn "mean = %A" (ie.Infer<Gaussian>(mean))
printfn "prec = %A" (ie.Infer<Gamma>(precision))

Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
mean = Gaussian(-0.09333, 0.007894)
prec = Gamma(51, 0.02484)[mean=1.267]
